In [1]:
%load_ext autoreload
%autoreload 2
import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import eq
import wandb

In [2]:
print(eq.data.default_catalogs_dir)

/home/zekai/repos/recast/data


In [5]:
catalog = eq.catalogs.ANSS_MultiCatalog(mag_completeness=2.0)

Generating the catalog...
Downloading...
train


In [9]:
print("Train sequence:")
print(catalog.train[0])
print("\nValidation sequence:")
print(catalog.val[0])
print("\nTest sequence:")
print(catalog.test[0])

Train sequence:
Sequence(
  inter_times: [102],
  arrival_times: [101],
  t_start: 0.0,
  t_end: 365.0,
  t_nll_start: 0.0,
  mag: [101],
  mag_bounds: [2],
  mag_nll_bounds: [2]
)

Validation sequence:
Sequence(
  inter_times: [89],
  arrival_times: [88],
  t_start: 0.0,
  t_end: 173.745,
  t_nll_start: 0.0,
  mag: [88],
  mag_bounds: [2],
  mag_nll_bounds: [2]
)

Test sequence:
Sequence(
  inter_times: [168],
  arrival_times: [167],
  t_start: 0.0,
  t_end: 365.0,
  t_nll_start: 0.0,
  mag: [167],
  mag_bounds: [2],
  mag_nll_bounds: [2]
)
